https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [1]:
import pickle

# load doc into memory
with open("cleaned_pairs.txt", 'rb') as f:
	paired_sent = pickle.load(f)

print(paired_sent[:5])

[['resumption of the session', 'hervatting van de zitting'], ['i declare resumed the session of the european parliament adjourned on friday december and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period', 'ik verklaar de zitting van het europees parlement die op vrijdag december werd onderbroken te zijn hervat ik wens u allen een gelukkig nieuwjaar en hoop dat u een goede vakantie heeft gehad'], ['although as you will have seen the dreaded millennium bug failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful', 'zoals u heeft kunnen constateren is de grote millenniumbug uitgebleven de burgers van een aantal van onze lidstaten zijn daarentegen door verschrikkelijke natuurrampen getroffen'], ['you have requested a debate on this subject in the course of the next few days during this partsession', 'u heeft aangegeven dat u deze vergaderperiode een debat wilt ov

In [2]:
class MakeVocab():
    def __init__(self) -> None:
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        self.num_words = 3
        self.num_sentences = 0
        self.longest_sentence = 0
    
    def add_word(self, word):
            if word not in self.word2index:
                # First entry of word into vocabulary
                self.word2index[word] = self.num_words
                self.word2count[word] = 1
                self.index2word[self.num_words] = word
                self.num_words += 1
            else:
                # Word exists; increase word count
                self.word2count[word] += 1

    def add_sentence(self, sentence):
            sentence_len = 0
            for word in sentence.split(' '):
                sentence_len += 1
                self. add_word(word)
            if sentence_len > self.longest_sentence:
                # This is the longest sentence
                self.longest_sentence = sentence_len
            # Count the number of sentences
            self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

In [3]:
vocab_eng = MakeVocab()
vocab_nl = MakeVocab()
for pair in paired_sent:
    vocab_eng.add_sentence(pair[0])
    vocab_nl.add_sentence(pair[1])
if vocab_eng.longest_sentence > vocab_nl.longest_sentence:
    longest_sentence = vocab_eng.longest_sentence
else:
    longest_sentence = vocab_nl.longest_sentence

In [4]:
print(vocab_eng.to_word(3))
print(vocab_nl.to_word(3))

resumption
hervatting


In [5]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

EOS_token = 1
SOS_token = 0

def indexesFromSentence(word2index, sentence):
    return [word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(1, -1)

def get_dataloader(batch_size):

    n = len(paired_sent)
    input_ids = np.zeros((n, longest_sentence + 1), dtype=np.int32)
    target_ids = np.zeros((n, longest_sentence + 1), dtype=np.int32)
    
    for index, (input, target) in enumerate(paired_sent):
        inp_ids = indexesFromSentence(vocab_eng.word2index, input)
        tgt_ids = indexesFromSentence(vocab_nl.word2index, target)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[index, :len(inp_ids)] = inp_ids
        target_ids[index, :len(tgt_ids)] = tgt_ids

    print(input_ids[:5])
    print(target_ids[:5])
    '''

    train_data = TensorDataset(torch.LongTensor(input_ids),
                               torch.LongTensor(target_ids))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return train_dataloader
    '''

get_dataloader(32)

: 